# A* algorithm - step by step
In the following, we develop the necessary code to implement the A* algorithm in C++.

## Store a grid in your program
In order to write the A* search algorithm, I'll need a grid to search through. 

Let's start with a hard-coded grid.

In [1]:
#include <iostream>
#include <vector>
using std::cout;
using std::vector;

vector<vector<int>> board = {
    {0, 1, 0, 0, 0, 0},
    {0, 1, 0, 0, 0, 0},
    {0, 1, 0, 0, 0, 0},
    {0, 1, 0, 0, 0, 0},
    {0, 0, 0, 0, 1, 0}
};

for (vector row : board) {
    for (int value : row) {
        cout << value << " ";
    }
    cout << "\n";
}

0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 0 0 0 1 0 


## Create a function to print the board: PrintBoard
Let's put the printing of the board into a function.

In [2]:
#include <iostream>
#include <vector>
using std::cout;
using std::vector;

In [3]:
// When I try using vector instead of std::vector, the
// compiler complains about that.
// TODO: Look into this in more detail and potentially raise an issue
// at: https://github.com/jupyter-xeus/xeus-cling/issues
void PrintBoard(const std::vector<std::vector<int>> board) {
    for (std::vector row : board) {
        for (int value : row) {
            cout << value << " ";
        }
        cout << "\n";
    }
}

In [4]:
PrintBoard(board)

0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 0 0 0 1 0 


## Create a function to read the board from a file: ReadBoardFile

In [5]:
#include <iostream>
#include <fstream>

In [6]:
void ReadBoardFile(const std::string &path) {
    std::ifstream board_file(path);
    if (board_file) {
        std::string line;
        while (getline(board_file, line)) {
            std::cout << line << "\n";
        }
    }
}

In [7]:
ReadBoardFile("files/1.board")

0,1,0,0,0,0,
0,1,0,0,0,0,
0,1,0,0,0,0,
0,1,0,0,0,0,
0,0,0,0,1,0,


## Create a function to parse lines into vectors: ParseLine

In [8]:
#include <sstream>

In [9]:
std::vector<int> ParseLine(const std::string &line) {
    // Assumption: Each line of the baord looks like this: 1, 0, 0, 0,
    std::vector<int> v;
    std::istringstream str_stream(line);
    
    char comma;
    int number;
    
    while (str_stream >> number >> comma && comma == ',') {
        v.push_back(number);
    }
    return v;
}

Let's update `ReadBoardFile` to use the `ParseLine` function.

In [10]:
using std::string;
using std::vector;
using std::ifstream;

In the following we need to use `auto` instead of `<vector<vector<int>>` because xeus-cling throws an error for the latter case. ([source](https://github.com/jupyter-xeus/xeus-cling/issues/40))

In [11]:
auto ReadBoardFile(const string &path) {
    std::ifstream board_file(path);
    std::vector<std::vector<int>> board;
    if (board_file) {
        std::string line;
        while (getline(board_file, line)) {
            board.push_back(ParseLine(line));
        }
    }
    return board;
}

In [12]:
std::vector<std::vector<int>> board = ReadBoardFile("files/1.board");
PrintBoard(board);

0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 1 0 0 0 0 
0 0 0 0 1 0 


## Formatting the Printed Board
The board will eventually have more than two cell states as the program becomes more complicated. Let's add formatting to improve the readability. We'll use formatting moving forward.

Let's print the board this way:
```
0   ⛰️   0   0   0   0
0   ⛰️   0   0   0   0
0   ⛰️   0   0   0   0
0   ⛰️   0   0   0   0
0   0    0   0  ⛰️   0
```

In [13]:
enum class State {kEmpty, kObstacle, kClosed, kPath};

In [14]:
string CellString(const State &state) {
    switch (state) {
        case State::kEmpty :
            return "0 ";
            break;
        case State::kObstacle:
            return "⛰️ ";
            break;
        case State::kClosed:
            return "x ";
            break;
        case State::kPath:
            return "🚗";
            break;
        default:
            std::cout << "Unknown State\n";
            throw std::runtime_error("Unknown State");
    }
}

In [15]:
void PrintBoard(const std::vector<std::vector<int>> board) {
    for (auto row : board) {
        for (int value : row) {
            cout << CellString(State(value));
        }
        cout << "\n";
    }
}

In [16]:
std::vector<std::vector<int>> board = ReadBoardFile("files/1.board");
PrintBoard(board);

0 ⛰️ 0 0 0 0 
0 ⛰️ 0 0 0 0 
0 ⛰️ 0 0 0 0 
0 ⛰️ 0 0 0 0 
0 0 0 0 ⛰️ 0 


# Putting it all together
Next, we're going to refactor all the functions to use `State` exclusively. Here is an overview over the API:

```Cpp
enum class State {kEmpty, kObstacle};

// Transform a state into a string
std::string CellString(const State &state) {...}

// Print the board to the standard output stream
void PrintBoard(const std::vector<std::vector<State>> board) {...}

// Transforms a line into a State vector
std::vector<State> ParseLine(const std::string &line) {...}

// Open the file containing the board, read the file line by line,
// transform 0 & 1 values to State values, return a 2D vector containing
// the board.
std::vector<std::vector<State>> ReadBoardFile(const std::string &path) {...}
```

In [17]:
#include <fstream>
#include <iostream>
#include <sstream>
#include <string>
#include <vector>
using std::cout;
using std::ifstream;
using std::istringstream;
using std::string;
using std::vector;

// Defined above, can't define it again else it throws an error
// enum class State {kEmpty, kObstacle, kClosed};

In [18]:
std::string CellString(const State &state) {
    switch (state) {
        case State::kEmpty :
            return "0 ";
            break;
        case State::kObstacle:
            return "⛰️ ";
            break;
        case State::kClosed:
            return "x ";
            break;
        case State::kPath:
            return "🚗";
            break;
        default:
            std::cout << "Unknown State\n";
            throw std::runtime_error("Unknown State");
    }
}

In [19]:
void PrintBoard(const std::vector<std::vector<State>> board) {
    for (std::vector<State> row : board) {
        for (State state : row) {
            std::cout << CellString(state) << " ";
        }
        std::cout << "\n";
    }
}

In [20]:
std::vector<State> ParseLine(const std::string &line) {
    std::vector<State> v;
    std::istringstream str_stream(line);
    
    int num;
    char comma;
    while(str_stream >> num >> comma && comma == ',') {
        v.push_back(State(num));
    }
    return v;
}

In [21]:
// auto refers to std::vector<std::vector<State>>
// have to use auto, else an error is thrown due to xeus-cling
auto ReadBoardFile(const std::string &path) {
    std::ifstream board_file(path);
    std::vector<std::vector<State>> board;
    if (board_file) {
        std::string line;
        while (getline(board_file, line)) {
            board.push_back(ParseLine(line));
        }
        return board;
    } else {
        throw std::runtime_error("File stream failed.");
    }
}

In [22]:
PrintBoard(ReadBoardFile("files/1.board"));

0  ⛰️  0  0  0  0  
0  ⛰️  0  0  0  0  
0  ⛰️  0  0  0  0  
0  ⛰️  0  0  0  0  
0  0  0  0  ⛰️  0  


## Summary
Content:
* Sending output to the terminal
* Variables & containers
  * Variable types
  * Vectors
  * auto
  * Define own types with enums
* Functions and control structures
  * Conditionals
  * Loops
  * Functions
* Data Input
  * Read data from a file
  * Parse data and process strings

Wrote a program that:
* Reads an input text file (board data)
* Parses, formats, and stores the data locally
  * Defined enum type to add ascii charachter formatting to the output
* Prints the output

# A* algorithms

Finds efficiently the path between two points in a grid.

The A* algorithms is an algorithm that's frequently used for path finding when working with graphs.

So far, we've developed code that reads board data from an input text file, parses, formats, and stores the data locally, and prints the board to the output.

In the following, we'll add a step between storing the data and printing the board:
* Reads an input text file (board data)
* Parses, formats, and stores the data locally
  * Defined enum type to add ascii charachter formatting to the output
* Find a path using A* search
* Prints the output

The A* algorithms is a discrete method for planning.

### General Discrete Path Search Algorithm
**Given:**
* Map
* Staring location
* Goal location
* Cost function

**Goal:**
* Find minimum cost path

**Method:**
* Keep a list of nodes to investigate further (expand)
* Keep track of which nodes were visited
* Keep track of the g-value (the number of steps taken so far)
* We always expand the node with the smallest g-value

### A* Algorithm
The difference between the A* algorithm and the general discrete path search algorithm is that A* uses a heuristic function that returns the number of steps it takes to get to the goal if there was no obstacle.

The heuristic function is therefore an optimistic guess of how far we are from the goal (because it assumes that there are no obstacles):

$$h(x,y) \leq \text{distance to goal from } x, y$$

The heuristic function is an underestimate or at best equal to the true distance from start to goal. It is an _admissible_ heuristic, meaning it never overestimates the cost of reaching the goal.

There are many valid functions for the heuristic functions, including setting everything to $0$, but that's not really useful here. The _Euclidean distance_ can be used to calculate $h(x, y)$.

#### Modifying the general discrete path search to the A* search
**Given:**
* Map
* Staring location
* Goal location
* Cost function
* Heuristic function $h(x, y)$

**Goal:**
* Find minimum cost path

**Method:**
* Keep a list of nodes to investigate further (expand)
* Keep track of which nodes were visited
* Keep track of the g-value (the number of steps taken so far)
* Keep track of the f-value: $f = g + h(x, y)$
* We always expand the node with the smallest f-value

#### A* Pseudocode
```
Search(grid, start_point, goal_point):
  1. Initialize an empty list of open nodes
  2. Initialize a starting node with the following:
    * x, y values given by start_point
    * g = 0 (cost for each move)
    * h (heuristic function, a function of the current coordinates and the goal)
  3. Add the new node to the list of open nodes.
  4. WHILE the list of open nodes is nonempty:
    1. Sort the open list by f-value
    2. Pop the optimal cell (called the current cell)
    3. Mark the cell's coordinates in the grid as part of the path
    4. IF the current cell is the goal cell:
      * return the grid
    5. ELSE expand the search to the currend node's neighbors:
      * Check each neighbor cell in the grid to ensure that the cell is empty: It has not been closed and is not an obstacle.
      * If the cell is empty, compute the cost (g value) and the heuristic, and add to the list of open nodes.
      * Mark the cell as closed.
  5. If the while loop exits because the list of open nodes is empty, then there are no new nodes to explore and there doesn't exist a path.
```

#### Summary
The A* algorithm finds a path from start node to end node by: 
* checking for open neighbors of the current node
* computing a heuristic for each of the neighbors
* adding those neighbors to the list of open nodes to explore next
* choosing the next node to explore with `min(g + h)`
Repeat the process until the goal is found, or until there are no open nodes to explore (meaning there is no path from start to end node).

### A* API description
* `main()`
  * `ReadBoardFile()` - Read the board data from a file
  * `Search()` - A* search function
    * WHILE open list not empty and next node not end goal:
      * `CellSort()` - Sort open list according to $h+g$ value
      * `ExpandNeighbors()` - Loop through the current node's neighbors and add valid neighbors to the open list. For each neighbor:
        * `CheckValidCell()` - Make sure the node coordinates are on the grid and that the grid cell is empty
        * `Heuristic()` - The [manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) to the goal
        * `AddToOpen()` - Add node to open list and mark grid cell as closed
  * `PrintSolution()` - Print the solution to the terminal with formatting
  
```cpp
std::string CellString(const State &state) { ... };
void PrintBoard(const std::vector<std::vector<State>> board) { ... };
std::vector<State> ParseLine(const std::string &line) { ... };
std::vector<std::vector<State>> ReadBoardFile(const std::string &path) { ... };

std::vector<std::vector<State>> Search(std::vector<std::vector<State>> board,
                                       int start[2], 
                                       int goal[2]) {...};
int Heuristic(const int x1,
              const int x2,
              const int y1,
              const int y2) { ... };
              
// Helper function for CellSort
bool Compare(std::vector<int> node1, std::vector<int> node2) { ... };

void CellSort(std::vector<std::vector<int>> &openlist) { ... };
void ExpandNeighbors(std::vector<std::vector<int>> &openlist,
                     int node[2]) { ... };
// Could make a copy of the board and change the values of visited nodes
// (e.g. to -1) to indicate that they've been already visited.
bool CheckValidCell(std::vector<std::vector<State>> board,
                    int node[2]) { ... };
void AddToOpen(const int x,
               const int y,
               const int g,
               const int h,
               std::vector<std::vector<int>> &openlist,
               std::vector<std::vector<State>> &board) { ... };

```
  
  
Let's get started! Taking the code we derived above and paste it in here:

In [23]:
#include <iostream>    // std::cout
#include <fstream>     // std::ifstream
#include <sstream>     // std::istringstream
#include <string>      // std::string
#include <vector>      // std::vector
#include <algorithm>   // std::sort
using std::cout;
using std::ifstream;
using std::istringstream;
using std::string;
using std::vector;


// Defined above, can't define it again else it throws an error
// enum class State {kEmpty, kObstacle, kClosed};

In [24]:
std::string CellString(const State &state) {
    switch (state) {
        case State::kEmpty :
            return "0 ";
            break;
        case State::kObstacle:
            return "⛰️ ";
            break;
        case State::kClosed:
            return "x ";
            break;
        case State::kPath:
            return "🚗";
            break;
        default:
            std::cout << "Unknown State\n";
            throw std::runtime_error("Unknown State");
    }
}

In [25]:
void PrintBoard(const std::vector<std::vector<State>> board) {
    for (std::vector<State> row : board) {
        for (State state : row) {
            std::cout << CellString(state) << " ";
        }
        std::cout << "\n";
    }
}

In [26]:
std::vector<State> ParseLine(const std::string &line) {
    std::vector<State> v;
    std::istringstream str_stream(line);
    
    int num;
    char comma;
    while(str_stream >> num >> comma && comma == ',') {
        v.push_back(State(num));
    }
    return v;
}

In [27]:
// auto refers to std::vector<std::vector<State>>
// have to use auto, else an error is thrown due to xeus-cling
auto ReadBoardFile(const std::string &path) {
    std::ifstream board_file(path);
    std::vector<std::vector<State>> board;
    if (board_file) {
        std::string line;
        while (getline(board_file, line)) {
            board.push_back(ParseLine(line));
        }
        return board;
    } else {
        throw std::runtime_error("File stream failed.");
    }
}

In [28]:
int Heuristic(const int p_x,
              const int p_y,
              const int q_x,
              const int q_y) {
    // Calculate the manhattan distance
    return abs(p_x - q_x) + abs(p_y - q_y);
}

In [29]:
void AddToOpen(int x,
               int y,
               int g,
               int h,
               std::vector<std::vector<int>> &openlist,
               std::vector<std::vector<State>> &board) {
    std::vector<int> node = {x, y, g, h};
    openlist.push_back(node);
    board[x][y] = State::kClosed;
}

In [30]:
// Helper function for CellSort
bool Compare(std::vector<int> node1, std::vector<int> node2) {
    // A node has these values: {x, y, g, h}
    int f1 = node1[2] + node1[3];
    int f2 = node2[2] + node2[3];
    return f1 > f2;
}

In [31]:
void CellSort(std::vector<std::vector<int>> &openlist) {
    std::sort (openlist.begin(), openlist.end(), Compare);
}

So, in the solution, CellSort is implemented this way:

```cpp
void CellSort(std::vector<std::vector<int>> *openlist) {
    std::sort (openlist->begin(), openlist->end(), Compare);
}
```

And then in Search called this way:
```cpp
CellSort(&openlist);
```

I understand that this version uses pointers as opposed to the reference to the `openlist`. But I don't understand the difference, nor why I would use one over the other.

In [32]:
bool CheckValidCell(std::vector<std::vector<State>> board,
                    int node[2]) {
    int x = node[0];
    int y = node[1];
    if (x < 0 || x > board.size() || 
        y < 0 || y > board[0].size() ||
        board[x][y] != State::kEmpty) {
        return false;
    }
    return true;
}

**Note:** This is where I stopped using xeus-cling exclusively to run my code. Calling `AddToOpen` in `ExpandNeighbors` kills the kernel, even though this code runs in a regular C++ file.

Maybe a todo for later, when I'm more comfortable with C++ and debugging it. The problem is also that xeus-cling doesn't give useful error messages, so it's hard to figure out what's going on.

In [33]:
void ExpandNeighbors(int x, int y, int g, const int goal[2],
                     std::vector<std::vector<int>> &openlist,
                     std::vector<std::vector<State>> &board) {
  g = g + 1;
  int h;

  // Four vectors for getting the nodes that are up, left, right,
  // and below current node
  std::vector<std::vector<int>> delta = {{0, 1}, {0, -1}, {1, 0}, {-1, 0}};

  int node[2];

  for (std::vector<int> d : delta) {
    node[0] = x + d[0];
    node[1] = y + d[1];
    h = Heuristic(node[0], node[1], goal[0], goal[1]);
    if (CheckValidCell(board, node)) {
      AddToOpen(node[0], node[1], g, h, openlist, board);
    }
  }
}

In [34]:
// auto refers to std::vector<std::vector<State>>
// have to use auto, else an error is thrown due to xeus-cling
auto Search(std::vector<std::vector<State>> &board, 
            int start[2], 
            int goal[2]) {
    std::vector<std::vector<int>> openlist {};
    int x = start[0];
    int y = start[1];
    int g = 0;
    int h = Heuristic(x, y, goal[0], goal[1]);
    AddToOpen(x, y, g, h, openlist, board);
    
    std::vector<int> current_node; 
    int jk = 0;
    while (!openlist.empty() && jk < 5) {
        jk += 1;
        std::cout << "-----\n";
        CellSort(openlist);
        current_node = openlist.back();
        openlist.pop_back();
        
        x = current_node[0];
        y = current_node[1];
        g = current_node[2];
        board[x][y] = State::kPath;

        if (x == goal[0] && y == goal[1]) {
            return board;
        }
        ExpandNeighbors(x, y, g, goal, openlist, board); 
    }
    return board;
}

In [35]:
PrintBoard(ReadBoardFile("files/1.board"));

0  ⛰️  0  0  0  0  
0  ⛰️  0  0  0  0  
0  ⛰️  0  0  0  0  
0  ⛰️  0  0  0  0  
0  0  0  0  ⛰️  0  


In [36]:
int start[2] {0, 0};
int goal[2] {4, 5};
std::vector<std::vector<State>> board = ReadBoardFile("files/1.board");

In [ ]:
std::vector<std::vector<State>> solution = Search(board, start, goal);

In [ ]:
PrintBoard(board);
cout << '\n';

# Working A* implementation

Since the last step of my implementation in Jupyter Notebook using xeus-cling doesn't work, I decided to go back to working in regular files. The working implementation is implemented in the file `astar.cpp`.